In [1]:
import os
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, Dropout, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
#'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [3]:
N_SPLITS = 5
N_STEPS = 20
N_DAYS = 60
DATA_DIR = f'data/processed_data/cv_data__{N_SPLITS}_class_id_folds__{N_STEPS}_steps__{N_DAYS}_days'

In [4]:
results = []

for i in range(N_SPLITS):

    # Load one partition of folds
    data = hkl.load(f'{DATA_DIR}/fold_{i}.hkl')
    #action_training_input = data['action_training_input']
    #recurrent_training_input = data['recurrent_training_input']
    prior_training_input = data['prior_training_input']
    completion_training_target = data['completion_training_target']
    problems_training_target = data['problems_training_target']
    training_target_sequence = data['training_target_sequence']
    #action_testing_input = data['action_testing_input']
    #recurrent_testing_input = data['recurrent_testing_input']
    prior_testing_input = data['prior_testing_input']
    completion_testing_target = data['completion_testing_target']
    problems_testing_target = data['problems_testing_target']
    testing_target_sequence = data['testing_target_sequence']
    
    # Clear session so models don't pile up
    keras.backend.clear_session()

    # Create model
    
    # Action Model
    #action_input_layer = Input(shape=action_training_input[0].shape, name='action')
    #action_model = Dropout(rate=0.5)(action_input_layer)
    #action_model = LSTM(units=64, return_sequences=False, activation='relu', dropout=0.5, recurrent_dropout=0.5)(action_model)
    
    # Recurrent Input
    #recurrent_input_layer = Input(shape=recurrent_training_input[0].shape, name='recurrent')
    #recurrent_model = Masking(mask_value=0.0)(recurrent_input_layer)
    #recurrent_model = Dropout(rate=0.5)(recurrent_model)
    #recurrent_model = LSTM(units=64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)(recurrent_model)
    
    # Prior Input
    prior_input_layer = Input(shape=prior_training_input[0].shape, name='prior')
    prior_model = Dropout(rate=0.5)(prior_input_layer)
    prior_model = Dense(units=64, activation='sigmoid')(prior_model)
    
    # Combined Model
    #model = Concatenate()([action_model, recurrent_model, prior_model])
    model = Dropout(rate=0.5)(prior_model)
    
    # Combined Output
    completion_output_layer = Dense(units=1, activation='sigmoid', name='completion')(model)
    problems_output_layer = Dense(units=1, activation='linear', name='problems')(model)
    
    combined_model = Model(prior_input_layer, [completion_output_layer, problems_output_layer])
    combined_model.compile(optimizer='adam', loss={'completion': 'binary_crossentropy', 'problems': 'mse'})

    # Train model
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    weights = {'completion': np.ones_like(completion_training_target) * 16, 'problems': completion_training_target}
    combined_model.fit(x={'prior': prior_training_input},
                       y={'completion': completion_training_target, 'problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    # Store model predictions
    completion_testing_output, problems_testing_output = combined_model.predict({'prior': prior_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    

pd.concat(results).to_csv('priors_cross_validation_results.csv', index=False)

Epoch 1/1000
3949/3949 [==============================] - 14s 3ms/step - loss: 16.0119 - completion_loss: 8.5847 - problems_loss: 7.4272 - val_loss: 14.5725 - val_completion_loss: 8.8408 - val_problems_loss: 5.7318
Epoch 2/1000
3949/3949 [==============================] - 13s 3ms/step - loss: 15.3597 - completion_loss: 8.2963 - problems_loss: 7.0634 - val_loss: 14.5201 - val_completion_loss: 8.8169 - val_problems_loss: 5.7032
Epoch 3/1000
3949/3949 [==============================] - 13s 3ms/step - loss: 15.2940 - completion_loss: 8.2763 - problems_loss: 7.0177 - val_loss: 14.5733 - val_completion_loss: 8.8567 - val_problems_loss: 5.7165
Epoch 4/1000
3949/3949 [==============================] - 13s 3ms/step - loss: 15.2199 - completion_loss: 8.2441 - problems_loss: 6.9758 - val_loss: 14.4875 - val_completion_loss: 8.7800 - val_problems_loss: 5.7075
Epoch 5/1000
3949/3949 [==============================] - 13s 3ms/step - loss: 15.2040 - completion_loss: 8.2561 - problems_loss: 6.9479 - v

3949/3949 [==============================] - 13s 3ms/step - loss: 14.6369 - completion_loss: 8.2189 - problems_loss: 6.4179 - val_loss: 14.1938 - val_completion_loss: 8.6514 - val_problems_loss: 5.5424
Epoch 16/1000
3949/3949 [==============================] - 13s 3ms/step - loss: 14.6577 - completion_loss: 8.2405 - problems_loss: 6.4172 - val_loss: 14.2247 - val_completion_loss: 8.6700 - val_problems_loss: 5.5547
Epoch 17/1000
3949/3949 [==============================] - 13s 3ms/step - loss: 14.6290 - completion_loss: 8.2277 - problems_loss: 6.4012 - val_loss: 14.1955 - val_completion_loss: 8.6513 - val_problems_loss: 5.5441
Epoch 18/1000
3949/3949 [==============================] - 13s 3ms/step - loss: 14.6233 - completion_loss: 8.2213 - problems_loss: 6.4020 - val_loss: 14.2062 - val_completion_loss: 8.6633 - val_problems_loss: 5.5429
Epoch 19/1000
3949/3949 [==============================] - 13s 3ms/step - loss: 14.6150 - completion_loss: 8.2160 - problems_loss: 6.3989 - val_loss: 

3949/3949 [==============================] - 15s 4ms/step - loss: 14.3813 - completion_loss: 8.2428 - problems_loss: 6.1385 - val_loss: 14.3267 - val_completion_loss: 8.5199 - val_problems_loss: 5.8069
Epoch 23/1000
3949/3949 [==============================] - 14s 4ms/step - loss: 14.3883 - completion_loss: 8.2555 - problems_loss: 6.1327 - val_loss: 14.3703 - val_completion_loss: 8.5410 - val_problems_loss: 5.8293
Epoch 24/1000
3949/3949 [==============================] - 15s 4ms/step - loss: 14.3874 - completion_loss: 8.2482 - problems_loss: 6.1392 - val_loss: 14.4449 - val_completion_loss: 8.6151 - val_problems_loss: 5.8297
Epoch 25/1000
3949/3949 [==============================] - 15s 4ms/step - loss: 14.3766 - completion_loss: 8.2468 - problems_loss: 6.1298 - val_loss: 14.3850 - val_completion_loss: 8.5503 - val_problems_loss: 5.8347
Epoch 26/1000
3949/3949 [==============================] - 14s 4ms/step - loss: 14.3770 - completion_loss: 8.2432 - problems_loss: 6.1338 - val_loss: 

3949/3949 [==============================] - 15s 4ms/step - loss: 14.9121 - completion_loss: 8.2067 - problems_loss: 6.7054 - val_loss: 14.4061 - val_completion_loss: 8.5714 - val_problems_loss: 5.8347
Epoch 24/1000
3949/3949 [==============================] - 15s 4ms/step - loss: 14.8997 - completion_loss: 8.2074 - problems_loss: 6.6923 - val_loss: 14.4077 - val_completion_loss: 8.5883 - val_problems_loss: 5.8194
Epoch 25/1000
3949/3949 [==============================] - 15s 4ms/step - loss: 14.8792 - completion_loss: 8.1957 - problems_loss: 6.6834 - val_loss: 14.3616 - val_completion_loss: 8.5424 - val_problems_loss: 5.8192
Epoch 26/1000
3949/3949 [==============================] - 15s 4ms/step - loss: 14.8995 - completion_loss: 8.2037 - problems_loss: 6.6958 - val_loss: 14.4421 - val_completion_loss: 8.6174 - val_problems_loss: 5.8247
Epoch 27/1000
3949/3949 [==============================] - 15s 4ms/step - loss: 14.8915 - completion_loss: 8.2119 - problems_loss: 6.6796 - val_loss: 

In [5]:
# Load the data

exp_norm_map = pd.read_csv('data/experiment_information/exp_norm_map.csv')
results = pd.read_csv('priors_cross_validation_results.csv')

In [6]:
# Evaluate the results

norm_exp_dict = defaultdict(lambda: 'None')
for n, e in zip(exp_norm_map['normal_id'], exp_norm_map['experiment_id']):
    norm_exp_dict[n] = e
results['experiment_sequence'] = results['target_sequence'].map(norm_exp_dict)

metrics = []
0
# Calculate the metrics for each sequence
for sequence, df in results.groupby('target_sequence'):
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([sequence, 
                    norm_exp_dict[sequence], 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for each fold
for fold, df in results.groupby('fold'):
    
    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([f'fold_{fold}', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for everything combined
completion_target = results['completion_target']
completion_prediction = results['completion_prediction']
completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_mse = mean_squared_error(completion_target, completion_prediction)

problems_target = results[results['completion_target'] == 1]['problems_target']
problems_prediction = results[results['completion_target'] == 1]['problems_prediction']
problems_mae = mean_absolute_error(problems_target, problems_prediction)
problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_mse = mean_squared_error(problems_target, problems_prediction)

metrics.append(['all_target_data', 
                'None', 
                len(results), 
                completion_auc, 
                completion_acc, 
                completion_r2, 
                completion_ev, 
                completion_mse, 
                problems_mae, 
                problems_mape, 
                problems_r2, 
                problems_ev, 
                problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['sequence_id', 
                                'experiment_id', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('priors_cross_validation_metrics.csv', index=False)

In [7]:
metrics.iloc[-6:]

,sequence_id,experiment_id,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
288,fold_0,None,42114,0.733250,0.762977,0.128157,0.128187,0.163227,1.928486,0.214565,0.027076,0.027178,7.364959
289,fold_1,None,42114,0.737779,0.756043,0.129186,0.129325,0.165776,1.976198,0.212980,0.022399,0.022493,9.328725
290,fold_2,None,42114,0.736913,0.759201,0.131706,0.131811,0.164680,2.026637,0.235152,0.022449,0.022538,10.012898
291,fold_3,None,42113,0.732057,0.755539,0.130634,0.131119,0.166984,1.943945,0.219360,0.029211,0.029359,7.655344
292,fold_4,None,42113,0.732295,0.755064,0.125639,0.125690,0.166322,1.953646,0.205624,0.028393,0.028524,7.906808
293,all_target_data,None,210568,0.734191,0.757765,0.129115,0.129159,0.165398,1.965762,0.217535,0.025733,0.025737,8.453210
